<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [5]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("AFDB.dta")
stata_df.head()

,period,period_start_day,value,_frequency,dataset_code,dataset_name,country,frequency,indicator,scale,units,indexed_at,provider_code,series_code,series_name,series_num
0,1991,1991-01-01,3874000.0,annual,bbkawjf,AFDB Socio Economic Database,AGO,A,LM.POP.EPP.TOT,1,thousands,2022-04-04T15:05:17.535Z,AFDB,LM.POP.EPP.TOT.AGO,"Total Employment, Population ('000) - Angola",Master
1,1992,1992-01-01,4015000.0,annual,bbkawjf,AFDB Socio Economic Database,AGO,A,LM.POP.EPP.TOT,1,thousands,2022-04-04T15:05:17.535Z,AFDB,LM.POP.EPP.TOT.AGO,"Total Employment, Population ('000) - Angola",Master
2,1993,1993-01-01,4206000.0,annual,bbkawjf,AFDB Socio Economic Database,AGO,A,LM.POP.EPP.TOT,1,thousands,2022-04-04T15:05:17.535Z,AFDB,LM.POP.EPP.TOT.AGO,"Total Employment, Population ('000) - Angola",Master
3,1994,1994-01-01,4334000.0,annual,bbkawjf,AFDB Socio Economic Database,AGO,A,LM.POP.EPP.TOT,1,thousands,2022-04-04T15:05:17.535Z,AFDB,LM.POP.EPP.TOT.AGO,"Total Employment, Population ('000) - Angola",Master
4,1995,1995-01-01,4461000.0,annual,bbkawjf,AFDB Socio Economic Database,AGO,A,LM.POP.EPP.TOT,1,thousands,2022-04-04T15:05:17.535Z,AFDB,LM.POP.EPP.TOT.AGO,"Total Employment, Population ('000) - Angola",Master


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [6]:
python_df = pd.read_pickle("AFDB.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,country,indicator,units,scale,frequency,Country,Indicator,Units,Scale,Frequency
0,annual,AFDB,bbkawjf,AFDB Socio Economic Database,FM.LBL.MONY.CN.AGO,Money (current LCU) - Angola,1995,1995-01-01,2689020.0,2.689020e+06,AGO,FM.LBL.MONY.CN,current-lcu,1,A,Angola,Money (current LCU),current LCU,1,Annual
1,annual,AFDB,bbkawjf,AFDB Socio Economic Database,FM.LBL.MONY.CN.AGO,Money (current LCU) - Angola,1996,1996-01-01,93917900.0,9.391790e+07,AGO,FM.LBL.MONY.CN,current-lcu,1,A,Angola,Money (current LCU),current LCU,1,Annual
2,annual,AFDB,bbkawjf,AFDB Socio Economic Database,FM.LBL.MONY.CN.AGO,Money (current LCU) - Angola,1997,1997-01-01,194910000.0,1.949100e+08,AGO,FM.LBL.MONY.CN,current-lcu,1,A,Angola,Money (current LCU),current LCU,1,Annual
3,annual,AFDB,bbkawjf,AFDB Socio Economic Database,FM.LBL.MONY.CN.AGO,Money (current LCU) - Angola,1998,1998-01-01,272432000.0,2.724320e+08,AGO,FM.LBL.MONY.CN,current-lcu,1,A,Angola,Money (current LCU),current LCU,1,Annual
4,annual,AFDB,bbkawjf,AFDB Socio Economic Database,FM.LBL.MONY.CN.AGO,Money (current LCU) - Angola,1999,1999-01-01,1206550000.0,1.206550e+09,AGO,FM.LBL.MONY.CN,current-lcu,1,A,Angola,Money (current LCU),current LCU,1,Annual


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [8]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [9]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [10]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [11]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True